In [1]:
from Katna.video import Video
from Katna.writer import KeyFrameDiskWriter
import os
import ntpath
import aiohttp, asyncio, nest_asyncio
import cv2
import time

nest_asyncio.apply()

In [2]:
videos_dir_path = os.path.join('.','video_test')
print(videos_dir_path)

.\video_test


In [3]:
class CustomDiskWriter(KeyFrameDiskWriter):
    # This method is used to generate output filename for a keyframe
    # Here we call the super on te base class and add a suffix
    def save_frame_data_to_disk(self, frame, file_name):
        """Saves the keyframe data to the disk at ouutput directory path set at
        initialization.
        :param frame: frame data to be written
        :type frame: np.array
        :param file_name: Name of the file
        :type file_name: str
        """

        file_full_path = os.path.join(self.output_dir_path, file_name + self.file_ext)
        # cv2.imwrite(file_full_path, frame)
        cv2.imencode('.jpg', frame)[1].tofile(file_full_path)

In [5]:
from func_timeout import func_set_timeout
import func_timeout


In [6]:
@func_set_timeout(30)#设定函数超执行时间_
def task(vd, video_file_path, no_of_frames_to_returned, diskwriter, file):
    print(video_file_path)
    vd.extract_video_keyframes(
    no_of_frames=no_of_frames_to_returned, file_path=video_file_path,
    writer=diskwriter)


In [7]:

# For windows, the below if condition is must.
if __name__ == "__main__":

  for file in os.listdir('video_test'):
    vd = Video()

    # number of images to be returned
    no_of_frames_to_returned = 1

    # initialize diskwriter to save data at desired location
    diskwriter = CustomDiskWriter(location="key_frames")
    video_file_path = os.path.join('.','video_test', file)
    try:
        task(vd, video_file_path, no_of_frames_to_returned, diskwriter, file)
    except func_timeout.exceptions.FunctionTimedOut:
        print('timeout')

.\video_test\1_big.mp4
timeout
.\video_test\2_small.mp4
timeout


# multithreads

In [1]:
import os
import subprocess
import cv2
import random
import threading
import os   
import time    
import random
import json

video_dict = {}

def iframes_ffmpeg(filename, save_path, i_frames):
    for frame_no in i_frames:
        image_name = os.path.splitext(filename)[0].split('/')[-1]
        outname = os.path.join(save_path, image_name + '_'+ str(frame_no + 1).zfill(4) + '.jpg')
        os.system('ffmpeg -skip_frame nokey -i {} -vf "select=eq(n\,{})" -vsync vfr {} '.format(filename, frame_no, outname))
        print(filename, str(frame_no))


def run_work(file, video_dict):
    keyframe_path = './frame_extracted/'
    command = 'ffprobe -v error -show_entries frame=pict_type -of default=noprint_wrappers=1'.split()
    out = subprocess.check_output(command + [file]).decode()
    f_types = out.replace('pict_type=', '').split()
    frame_types = zip(range(len(f_types)), f_types)
    i_frames = [x[0] for x in frame_types if x[1] == 'I']
    # K = min(3, len(i_frames))
    # keyframe_id_set = list(random.sample(i_frames, K))
    video_dict[file] = i_frames
    # if keyframe_id_set:
    #     iframes_ffmpeg(file, keyframe_path, keyframe_id_set)
    # print('finish')

def main():

    video_path = './video_content/'

    video_path_list = os.listdir(video_path)
    count = 18000
    video_dict = {}
    while count < len(video_path_list):
        threads = []
        for file in video_path_list[count:count+12]:
            file = os.path.join(video_path, file)
            print(file)
            count+=1
            th = threading.Thread(target=run_work, args=(file,video_dict,))
            th.start()
            threads.append(th)
        for th in threads:
            th.join()
        if count%1200==0 or count==len(video_path_list):
            with open('./video_keyframes_list/video_keyframes_dict_{}.json'.format(count), 'w', encoding='utf-8') as f:
                json.dump(video_dict, f, ensure_ascii=False)
            video_dict = {}

    # print(count)
    # if count-12 > 0:
    #     else_video_apth_list = video_path_list[count-12:]
    #     # video_dict = {}
    #     threads = []
    #     for file in else_video_apth_list:
    #         file = os.path.join(video_path, file)
    #         print(file)
    #         th = threading.Thread(target=run_work, args=(file,video_dict,))
    #         th.start()
    #         threads.append(th)
    #     for th in threads:
    #         th.join()
    #     with open('./video_keyframes_list/video_keyframes_dict_final.json', 'w', encoding='utf-8') as f:
    #             json.dump(video_dict, f, ensure_ascii=False)

if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        print('jump it')
    # with open('video_keyframes_dict.json', 'w', encoding='utf-8') as f:
    #     json.dump(video_dict, f, ensure_ascii=False)

./video_content/7107247252837616930.mp4
./video_content/7107303283013815591.mp4
./video_content/7107385623144615205.mp4
./video_content/7107395360032640268.mp4
./video_content/7107404592710815012.mp4
./video_content/7107414888733101319.mp4
./video_content/7107417291670736141.mp4
./video_content/7107419181984828703.mp4
./video_content/7107426972350893347.mp4
./video_content/7107429833310784772.mp4
./video_content/7107431984934243592.mp4
./video_content/7107435164753497385.mp4
./video_content/7107439603329633566.mp4
./video_content/7107440225374244132.mp4
./video_content/7107452690870914341.mp4
./video_content/7107453745692298537.mp4
./video_content/7107457012547554574.mp4
./video_content/7107459626265775400.mp4
./video_content/7107465630076407076.mp4
./video_content/7107467190936341794.mp4
./video_content/7107478113721879815.mp4
./video_content/7107484928266210560.mp4
./video_content/7107488810153315614.mp4
./video_content/7107493053790833952.mp4
./video_content/7107496065036717349.mp4


In [3]:
v_d = {}
run_work('./video_test/7038580844239260958.mp4',video_dict=v_d)
v_d

{'./video_test/7038580844239260958.mp4': [0,
  225,
  461,
  634,
  782,
  942,
  972,
  1095,
  1199,
  1332,
  1429,
  1679,
  1865,
  1985,
  2177]}

In [1]:
import os
import subprocess
import cv2
import random
import threading
import os   
import time    
import random
import json
import traceback

def iframes_ffmpeg(filename, save_path, i_frames):
    threads = []
    for frame_no in i_frames:
        image_name = os.path.splitext(filename)[0].split('/')[-1]
        # outname = os.path.join(save_path, image_name + '_'+ str(frame_no + 1).zfill(4) + '_' + '%d'+ '.jpg')
        outname = os.path.join(save_path, image_name + '_'+ str(frame_no + 1).zfill(4) + '.jpg')
        th = threading.Thread(target=run_ffmpeg, args=(filename, outname, frame_no,))
        th.start()
        threads.append(th)
    for th in threads:
        th.join()

def run_ffmpeg(filename, outname, frame_no):
        # os.system('ffmpeg -skip_frame nokey -i {} -vf "select=eq(n\,{})" -vsync vfr {} '.format(filename, frame_no, outname))
        os.system('ffmpeg -i {} -vf "select=eq(n\,{})" -vframes 1 {} -vsync 0 -y '.format(filename, frame_no, outname))
        # os.system('ffmpeg -y -vsync 0 -c:v h264_cuvid -i {} -vf "select=eq(n\,{})" {} '.format(filename, frame_no, outname))
        print(filename, str(frame_no))


def main():
        
    key_frame_list = os.listdir('./video_keyframes_list')
    all_key_frame_dict = {}
    for file in key_frame_list:
        with open(os.path.join('./video_keyframes_list', file), 'r', encoding='utf-8') as f:
            video_frame_dict = json.load(f)
        all_key_frame_dict.update(video_frame_dict)
    
    # dict_key = list(all_key_frame_dict.keys())
    # count = 0
    
    # while count < len(dict_key):
    #     threads = []
    #     for file in dict_key[count:count+12]:
    #         frame_list = all_key_frame_dict[file]
    #         K = min(3, len(frame_list))
    #         if K<3:
    #             three_keyframe_list = set(frame_list[0], frame_list[len(frame_list)//2], frame_list[-1])
    #         else:
    #             three_keyframe_list = list(random.sample(frame_list, K))
    #         count+=1
    #         # print(file, )
    #         th = threading.Thread(target=iframes_ffmpeg, args=(file, './frame_extracted', three_keyframe_list,))
    #         th.start()
    #         threads.append(th)
    #     for th in threads:
    #         th.join()
    for video_file_name, frame_list in all_key_frame_dict.items():
        try:
            K = min(3, len(frame_list))
            if K<3:
                three_keyframe_list = set([frame_list[0], frame_list[len(frame_list)//2], frame_list[-1]])
            else:
                three_keyframe_list = list(random.sample(frame_list, K))
            print(video_file_name)
            iframes_ffmpeg(video_file_name, './frame_extracted', three_keyframe_list)
        except Exception as e:
            print(video_file_name)
            traceback.print_exc()

if __name__ == '__main__':
    main()

./video_content/7015438878903569699.mp4
./video_content/7015438878903569699.mp4 128
./video_content/7015438878903569699.mp4 640
./video_content/7015438878903569699.mp4 896
./video_content/7015426956145560847.mp4
./video_content/7015426956145560847.mp4 250
./video_content/7015426956145560847.mp4 652
./video_content/7015426956145560847.mp4 1466
./video_content/7015405766815304968.mp4
./video_content/7015405766815304968.mp4 250
./video_content/7015405766815304968.mp4 500
./video_content/7015405766815304968.mp4 919
./video_content/7015450160083815715.mp4
./video_content/7015450160083815715.mp4 256
./video_content/7015450160083815715.mp4 768
./video_content/7015450160083815715.mp4 1024
./video_content/7015438265566121251.mp4
./video_content/7015438265566121251.mp4 640
./video_content/7015438265566121251.mp4 896
./video_content/7015438265566121251.mp4 1280
./video_content/7015446243010809101.mp4
./video_content/7015446243010809101.mp4 578
./video_content/7015446243010809101.mp4 828
./video_c

In [2]:
d = {1:2, 3:4}
list(d.values())

[2, 4]

In [1]:
os.system('ffmpeg -i ./video_content/7015438878903569699.mp4 -vf "select=eq(n\,1024)" -vframes 1 ./video_test/fuck.jpg -vsync 0 -y')

0

In [ ]:
import os
 
dir_name = './video_content/'
file_list = os.listdir(dir_name)
 
for file in file_list:
    try:
        old_file_name = os.path.join(dir_name, file)
        new_file_name = os.path.join(dir_name, file.split('.')[0].split('_')[1]+'.mp4')
        os.rename(old_file_name,new_file_name)
    except:
        print(file)

